In [60]:
import os
import cdsapi

import json
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotnine as p9
import seaborn as sns

In [61]:
#!pip install sdcpy
from tqdm.auto import tqdm
from collections import defaultdict
from sdcpy.scale_dependent_correlation import SDCAnalysis
import warnings

## SDC Analysis for groups of contiguous countries

In [62]:
world = pd.read_pickle('../data/world_shape.pkl')

In [63]:
country_to_iso = world[['country', 'iso3']].drop_duplicates()

In [64]:
covid_df = pd.read_csv('../data/covid/covid19_world.csv')

In [65]:
country_groups = pd.read_csv('../data/country_groups.csv')

In [66]:
lon_lat_country = pd.read_csv('../data/coords_region_0.5.csv').merge(country_to_iso)

In [67]:
lon_lat_group = lon_lat_country.merge(country_groups)

In [68]:
processed_path = "../data/climate/grid_0.5/processed/"
relevant_files = [f for f in os.listdir(processed_path) if (f[:9] >= '2021_12_0') & (f[:9] <= '2022_12_2')]
climate_df = pd.concat([pd.read_pickle(f'{processed_path}{f}') for f in relevant_files])

In [69]:
if not os.path.exists('../data/processed_05_grid.csv'):
    dens = (
        xr.load_dataset("../data/gpw_v4_population_density_rev11_2pt5_min.nc")
        .sel({"raster": 4})
        .to_dataframe()
        .dropna()
        .reset_index()
        .drop(columns="raster")
        .rename(
            columns={
                "Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 2.5 arc-minutes": "density"
            }
        )
        # Rounding to closest .5 degree
        .assign(latitude=lambda dd: (dd.latitude * 2).round() / 2)
        .assign(longitude=lambda dd: (dd.longitude * 2).round() / 2)
        .groupby(["latitude", "longitude"])
        .mean()
        .reset_index()
    )

    dens.to_csv('../data/processed_05_grid.csv', index=False)
dens = pd.read_csv("../data/processed_05_grid.csv")

In [70]:
f = (world
     .merge(country_groups)
     .pipe(lambda dd: p9.ggplot(dd)
       + p9.geom_map(data=world, fill='gray')
       + p9.geom_map(p9.aes(fill='group'))
       + p9.ylim(-55, None)
       + p9.labs(fill='')
       + p9.theme_void()
       + p9.theme(figure_size=(9, 4), dpi=200, legend_key_height=15)
      )
)
f.save("../results/world_plot.png")

c:\Users\brand\anaconda3\envs\tfm_env\Lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 9 x 4 in image.
c:\Users\brand\anaconda3\envs\tfm_env\Lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../results/world_plot.png


In [71]:
weighted_cells = (dens.merge(lon_lat_group)
    .groupby('group', as_index=False)
    .apply(lambda gdd: gdd.assign( relative_weight=lambda gdd: gdd.density / gdd.density.sum()))
    .reset_index(drop=True)
)

C:\Users\brand\AppData\Local\Temp\ipykernel_18988\1766496956.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [72]:
weighted_climate_df = (
        weighted_cells.dropna()
        .merge(climate_df, on=["latitude", "longitude"])
        .assign(temperature=lambda dd: dd.temperature * dd.relative_weight,
                absolute_humidity=lambda dd: dd.absolute_humidity * dd.relative_weight)
        .groupby(['group', "date"], as_index=False)
        [["temperature", "absolute_humidity"]]
        .agg({"temperature": "sum", "absolute_humidity": "sum"})
)

In [ ]:
groups_covid_ts = \
(covid_df
 .merge(country_groups)
 .assign(date=lambda dd: pd.to_datetime(dd.date))
 .groupby('group')
 .apply(lambda dd: 
        dd.set_index('date')
        .resample('D')
        .sum()
        ['new_cases']
        #.rolling(center=True, window=7)
        #.mean()
        .loc['2021-12-01': '2022-12-31'])
)

C:\Users\brand\AppData\Local\Temp\ipykernel_23900\1085163371.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [17]:
groups_weather_ts = \
(weighted_climate_df
 .assign(date=lambda dd: pd.to_datetime(dd.date))
 .query('"2021-12-01" <= date <= "2022-12-31"')
 .set_index(['group', 'date'])
)

In [21]:
CV_LABELS = {'absolute_humidity': 'Absolute Humidity [g/m³]',
             'temperature': 'Temperature [°C]',
             'relative_humidity': 'Relative Humidity [%]',
             'total_precipitation': 'Total Precipitation [mm]'}

In [23]:
import openpyxl

w = 105
out_dir = '../results'
for group in tqdm(country_groups.group.unique(), desc='Processing Groups', leave=False):
    for weather_variable in ['temperature', 'absolute_humidity']:
        covid_ts = groups_covid_ts.loc[group]
        weather_ts = groups_weather_ts.loc[group][weather_variable]
        sdc = SDCAnalysis(ts1=weather_ts,
                          ts2=covid_ts,
                          fragment_size=w,
                          method='spearman',
                          max_lag=0,
                          min_lag=-21)
        sdc.to_excel(f"{out_dir}/tables/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.xlsx")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sdc.combi_plot(dpi=120,
                           figsize=(7, 7),
                           title=f"SDC with s={w}: \nCOVID19 daily confirmed cases in" +
                                 f" {group} ~ {weather_variable}",
                           xlabel=CV_LABELS[weather_variable],
                           ylabel=f"Confirmed COVID19 cases in {group}",
                           alpha=0.05,
                           max_lag=0,
                           min_lag=-21,
                           max_r=1,
                           wspace=.35,
                           hspace=.35)
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.png")
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{group}_{weather_variable}.pdf")

## SDC Analysis for European countries

In [79]:
eur_countries = lon_lat_country[lon_lat_country['country'].isin(['Germany', 'Spain', 'France', 'United Kingdom', 'Italy'])]

In [124]:
eur_covid_ts = (
    covid_df
    .query("country in ['Germany', 'Spain', 'France', 'United Kingdom', 'Italy']")
    .assign(date=lambda dd: pd.to_datetime(dd['date']))
    .set_index('date')
    .sort_index()
    [['country','new_cases']]
    .loc['2021-12-01':'2022-12-31']
)

In [139]:
eur_weighted_cells = (dens.merge(lon_lat_country)
    .groupby('country', as_index=False)
    .apply(lambda gdd: gdd.assign( relative_weight=lambda gdd: gdd.density / gdd.density.sum()))
    .reset_index(drop=True)
)

C:\Users\brand\AppData\Local\Temp\ipykernel_23900\3096511853.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [160]:
eur_weighted_climate_df = (
        eur_weighted_cells.dropna()
        .merge(climate_df, on=["latitude", "longitude"])
        .assign(temperature=lambda dd: dd.temperature * dd.relative_weight,
                absolute_humidity=lambda dd: dd.absolute_humidity * dd.relative_weight)
        .groupby(["date","country_x"], as_index=False)
        [["temperature", "absolute_humidity"]]
        .agg({"temperature": "sum", "absolute_humidity": "sum"})
)

In [164]:
eur_weather_ts = (
    eur_weighted_climate_df
    .query("country_x in ['Germany', 'Spain', 'France', 'United Kingdom', 'Italy']")
    .assign(date=lambda dd: pd.to_datetime(dd['date']))
    .query('"2021-12-01" <= date <= "2022-12-31"')
    .set_index('date')
)

In [167]:

w = 105
out_dir = '../results'
for country in tqdm(['Germany', 'Spain', 'France', 'United Kingdom', 'Italy'], desc='Processing Countries', leave=False):
    for weather_variable in ['temperature', 'absolute_humidity']:
        country_covid_ts = eur_covid_ts[eur_covid_ts['country'] == country]['new_cases']
        country_weather_ts = eur_weather_ts[eur_weather_ts['country_x'] == country][weather_variable]
        sdc = SDCAnalysis(ts1=country_weather_ts,
                          ts2=country_covid_ts,
                          fragment_size=w,
                          method='spearman',
                          max_lag=0,
                          min_lag=-21)
        sdc.to_excel(f"{out_dir}/tables/SDC_eur_countries/sdc_{w}_{country}_{weather_variable}.xlsx")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sdc.combi_plot(dpi=120,
                           figsize=(7, 7),
                           title=f"SDC with s={w}: \nCOVID19 daily confirmed cases in" +
                                 f" {country} ~ {weather_variable}",
                           xlabel=CV_LABELS[weather_variable],
                           ylabel=f"Confirmed COVID19 cases in {country}",
                           alpha=0.05,
                           max_lag=0,
                           min_lag=-21,
                           max_r=1,
                           wspace=.35,
                           hspace=.35)
        plt.savefig(f"{out_dir}/figures/SDC_eur_countries/sdc_{w}_{country}_{weather_variable}.png")
        plt.savefig(f"{out_dir}/figures/SDC_eur_countries/sdc_{w}_{country}_{weather_variable}.pdf")
        plt.close()

## SDC Analysis for EU regions

In [82]:
lon_lat_region = pd.read_csv('../data/coords_region_EU_regions.csv')

In [83]:
covid_eu = pd.read_csv('../data/covid/covid_eu.csv')

In [84]:
processed_path = "../data/climate/grid_0.25/processed/"
relevant_files = [f for f in os.listdir(processed_path) if (f[:9] >= '2021_12_0') & (f[:9] <= '2022_12_2')]
eu4_weather = pd.concat([pd.read_pickle(f'{processed_path}{f}') for f in relevant_files])

In [85]:
dens = (
    xr.load_dataset("../data/gpw_v4_population_density_rev11_2pt5_min.nc")
    .sel({"raster": 4})
    .to_dataframe()
    .dropna()
    .reset_index()
    .drop(columns="raster")
    .rename(
        columns={
            "Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 2.5 arc-minutes": "density"
        }
    )
    # Here I am first, rounding up to .25 resolution and then shifting the map so that longitudes
    # are on the -180 +180 domain. 
    .assign(latitude=lambda dd: (dd.latitude * 4).round() / 4)
    .assign(longitude=lambda dd: (dd.longitude * 4).round() / 4)
    .groupby(["latitude", "longitude"])
    .mean()
    .reset_index()
)

In [87]:
weighted_cells = (
    dens.merge(lon_lat_region)
    .groupby('region', as_index=False)
    .apply(
        lambda gdd: gdd.assign(
            relative_weight=lambda gdd: gdd.density / gdd.density.sum()
        )
    )
    .reset_index(drop=True)
)

C:\Users\brand\AppData\Local\Temp\ipykernel_18988\433515362.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [88]:
weighted_climate_df = (
    weighted_cells
    .dropna()
    .merge(eu4_weather.merge(lon_lat_region), on=["latitude", "longitude", "region"])
    .assign(
        temperature=lambda dd: dd.temperature * dd.relative_weight,
        absolute_humidity=lambda dd: dd.absolute_humidity * dd.relative_weight
    )
    .groupby(['region', "date"])
    .agg({"temperature": "sum",
          "absolute_humidity": "sum",
          "relative_humidity": "sum",
          "total_precipitation": "sum"})
    .reset_index()
    .assign(region=lambda dd: dd.region.replace({'Valle d’Aosta/Vallée d’Aoste': "Valle d'Aosta"}))
    .melt(['region', 'date'])
    .assign(region=lambda dd: dd.region.str.replace('/', '-'))
    .sort_values(['region', 'date'])
    
)

In [94]:
weighted_climate_df

,region,date,variable,value
0,Abruzzo,2021-12-01,temperature,8.593485
14256,Abruzzo,2021-12-01,absolute_humidity,6.152578
28512,Abruzzo,2021-12-01,relative_humidity,17.662364
42768,Abruzzo,2021-12-01,total_precipitation,54.509184
1,Abruzzo,2021-12-02,temperature,11.207309
...,...,...,...,...
57022,Veneto,2022-12-30,total_precipitation,81.066952
14255,Veneto,2022-12-31,temperature,8.384002
28511,Veneto,2022-12-31,absolute_humidity,7.834230
42767,Veneto,2022-12-31,relative_humidity,36.941367


In [118]:
covid_sdc_df = (covid_eu
                .query('"2021-12-01" <= date <= "2022-12-31"')
                .loc[lambda dd: dd.region.isin(weighted_climate_df.region.unique())]
)
with open('../data/regions_country.json', 'r') as json_file:
    regions_country = json.load(json_file)

In [119]:
covid_sdc_df

,date,iso3,region,new_cases
647,2021-12-01,DEU,Baden-Württemberg,11589.0
648,2021-12-02,DEU,Baden-Württemberg,10735.0
649,2021-12-03,DEU,Baden-Württemberg,10587.0
650,2021-12-04,DEU,Baden-Württemberg,6487.0
651,2021-12-05,DEU,Baden-Württemberg,3317.0
...,...,...,...,...
49992,2022-12-27,ITA,Veneto,763.0
49993,2022-12-28,ITA,Veneto,4214.0
49994,2022-12-29,ITA,Veneto,3197.0
49995,2022-12-30,ITA,Veneto,2991.0


In [120]:
w = (weighted_climate_df
                 .query(f'region=="Veneto" & variable=="temperature"')
                 .set_index('date')
                 .value)

In [121]:
c = (covid_sdc_df
                 .query(f'region=="Veneto"')
                 .set_index('date')
                 .new_cases
                 .rolling(center=True, window=7)
                 .mean()
                 .dropna())

In [122]:
w.index = pd.to_datetime(w.index)
c.index = pd.to_datetime(c.index)   

In [123]:
sdc = SDCAnalysis(w, c, fragment_size=21, min_lag=-21, max_lag=0, method='spearman')

In [124]:
a = sdc.get_ranges_df(bin_size=3, min_bin=-6, max_bin=32)

KeyError: 'cat_value'

In [91]:
ranges_df = []
errors_list = {}
window = 21
sdcs = {}
out_dir_tab = '../results/tables/SDC_EU'
out_dir_fig = '../results/figures/SDC_EU'

for region in tqdm(covid_sdc_df.region.unique(), leave=False, desc='Regions'):
    country = regions_country[region]
    for var in tqdm(['temperature', 'absolute_humidity'], leave=False, desc='Weather variables'):
            os.makedirs(f"{out_dir_fig}/{country}/window_21/{region}", exist_ok=True)
            os.makedirs(f"{out_dir_fig}/{country}/window_21/{region}", exist_ok=True)
            
            w = (weighted_climate_df
                 .query(f'region=="{region}" & variable=="{var}"')
                 .set_index('date')
                 .value)

            c = (covid_sdc_df
                 .query(f'region=="{region}"')
                 .set_index('date')
                 .new_cases
                 .rolling(center=True, window=7)
                 .mean()
                 .dropna())
            
            # Ensure the indices are in datetime format
            w.index = pd.to_datetime(w.index)
            c.index = pd.to_datetime(c.index)   
        
            bin_size = 2 if var == 'absolute_humidity' else 3
            max_bin = 22 if var == 'absolute_humidity' else 32
            min_bin = 0 if var == 'absolute_humidity' else -6
            sdc = SDCAnalysis(w, c, fragment_size=window, min_lag=-21, max_lag=0, method='spearman')
            sdcs[f'{region}_{var}'] = sdc

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                try:
                    ranges_df.append(sdc.get_ranges_df(bin_size=bin_size, min_bin=min_bin, max_bin=max_bin)
                                      .assign(region=region, var=var, country=regions_country[region]
                                              ))

                except Exception as e:
                    errors_list[f'{region}_{var}'] = e

Regions:   0%|          | 0/35 [00:00<?, ?it/s]






















































































































































































































































































































































































































































































































































































































Regions:   3%|▎         | 1/35 [01:30<51:21, 90.62s/it]












































































































































































































































































































KeyboardInterrupt: 